<a href="https://colab.research.google.com/github/roni762583/TopBots_python/blob/main/TopBotsPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TopBots
# make pd df from json
Candles_H_df = json_to_pandas(response2)

# start analysind Candles_H_df and get ready to feed into NN predict High, and Low of next hourly bar
# part 1 of top bots

# get price inflection locations X 1/hours ago
inflections = list()
inflection = ''
n = 0
c = 0
for indx in Candles_H_df.index:
  if (n<=2):
    n+=1
    continue
  h2 = Candles_H_df['high'][n-2] # starts at index 0
  h1 = Candles_H_df['high'][n-1]
  h = Candles_H_df['high'][n]
  
  l2 = Candles_H_df['low'][n-2] # starts at index 0
  l1 = Candles_H_df['low'][n-1]
  l = Candles_H_df['low'][n]

  if(h1>h and h1>h2):
    #print(inflections[-1])
    #if(inflections[-1][1] == 'H' and inflections[-1][2]<h1): # fix this to only record highest high in consecutive 'H's
    inflection = 'H'
    
  if(l1<l and l1<l2):
    if not inflection:
      inflection = 'L'
    else:
      # supposed to ensure inflections alternate 'H' and 'L'
      inflection = 'H' if (inflections[-1][1] == 'L') else 'L'
      #c+=1
      #print(c) # it works 68 times
  
  price = Candles_H_df['high'][n-1] if(inflection=='H') else Candles_H_df['low'][n-1]
  if inflection:
    inflections.append( (Candles_H_df.index[n]+pd.DateOffset(hours=-1) ,inflection,price) ) #time index is 1 hr ahead
    #print(Candles_H_df.index[n])
    #print(Candles_H_df.index[n]+pd.DateOffset(hours=-1))
    #print(indx, inflection, price)
  
  n+=1
  inflection = ''

# part 2 - clean up repeating inflections
inflectionsToDelete = list()
for i in range(1,len(inflections)-1):
  #print (i, '   ', inflections[i][1])
  if(inflections[i][1] ==  inflections[i-1][1]): #if repeating inflection found, get rid of consecitive 'L's
    #print (i, inflections[i][1], inflections[i-1][1])
    if(inflections[i][1] == 'L'):
      #print (type(inflections[i][2]))
      if(inflections[i][2] > inflections[i-1][2]): # if new inflection is lower low, choose it instead of prev. inflection
        #print (i, inflections[i][1], inflections[i-1][1])
        inflectionsToDelete.append(inflections[i])

    # get rid of consecitive 'H's
    if(inflections[i][1] == 'H'):
      
      if(inflections[i][2] < inflections[i-1][2]): # if new inflection is higher high, choose it instead of prev. inflection
        inflectionsToDelete.append(inflections[i])
# some repeating inflections, but many have been removed, this can be further debugged later
print('len(inflections) = ', len(inflections))
inflections = [x for x in inflections if x not in inflectionsToDelete]
print('len(inflections) after delete ', len(inflections))